# Pruebas

## Voxel size

In [ ]:
import os
import numpy as np
import nibabel as nb
from skimage.measure import regionprops, label
from matplotlib import pyplot as plt
import math

METHOD = 'reg' # reg, dl, manual
LABEL = 'lens+globe' # eyeball, lens+globe

# Paths - segmentation results
if METHOD=='reg':
    main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
    input_path = main_path + 'non_labeled_dataset_nifti_reg_2/'
elif METHOD=='dl':
    main_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/3D_multilabel/experiment_0/'
    input_path = main_path + 'test_orig_reg-cropped_non-labeled/'
elif METHOD=='manual':
    main_path = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/'
    input_path = main_path + 'a123/'

# List of volumes for each individual label and subject
len_path = len([elem for elem in os.listdir(input_path)])
name_subject = [None]*len_path
axial_length_total = np.zeros(len_path)
outliers = list()

i=0
for folder1 in sorted(os.listdir(input_path)):

    # Load image in array format
    if METHOD=='reg': lab_arr = nb.load(f'{input_path}{folder1}/labels.nii.gz').header.get_zooms()
    elif METHOD=='dl': lab_arr = nb.load(f'{input_path}{folder1}').get_fdata()
    elif METHOD=='manual': lab_arr = nb.load(f'{input_path}{folder1}/input/{folder1}_labels_cropped.nii.gz').get_fdata()

    print(f'voxel size = ({lab_arr})')

    i+=1
    if i==1:
        break

## Sublist from list

In [ ]:
import SimpleITK as sitk
import sys, os, glob
import pydicomext as pydcm
from pydicom.fileset import FileSet
from deid.data import get_dataset
from deid.dicom import get_files, get_identifiers
from pathlib import Path
import numpy as np

input_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset/'

# List of volumes for each individual label and subject
len_path = len([elem for elem in os.listdir(input_path)])
name_subject = [None]*len_path
age_subject = [None]*len_path
sex_subject = [None]*len_path
height_subject = [None]*len_path
weight_subject = [None]*len_path
bmi_subject = [None]*len_path

# Create sorted list
i=0
list_subs = [None]*len_path
for folder1 in sorted(os.listdir(input_path)):
    list_subs[i] = folder1
    i+=1

index_sub = list_subs.index('2022160103683')
print(index_sub)

# Loop sublist
k=0
for j in range(index_sub, len_path):
    k+=1
print(k)    

## itkwidgets

imports and paths

In [ ]:
import nibabel as nb
import numpy as np
import scipy.misc
import itk, itkwidgets
from itkwidgets import view

ima_file = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/non_labeled_dataset_nifti_nnunet/AEye_2022160100001_0000.nii.gz'
lab_file = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing/AEye_2022160100001.nii.gz'

nibabel

In [ ]:
# ITKWIDGETS
# view(ima_arr[:, :, slices[0]:slices[len(slices)-1]])
# ima_itk = itk.imread(f'{input_path}{folder1}')
# view(ima_arr_read)


# ima_nb = nb.load(f'{ima_path}{ima_file}')
# ima_arr = np.array(ima_nb.get_fdata()) # image matrix
# ima_arr_uint8 = ima_arr.astype(np.uint8)

# view(ima_arr)

lab_nb = nb.load(f'{lab_path}{lab_file}')
lab_arr = np.array(lab_nb.get_fdata()) # image matrix
lab_arr_uint8 = lab_arr.astype(np.uint8)

view(lab_arr)

itk

In [ ]:
lab_itk = itk.imread(f'{lab_path}{lab_file}')

view(lab_itk)

In [ ]:
# Segment the cells
smoothed = itk.smoothing_recursive_gaussian_image_filter(lab_itk)

thresholded = itk.moments_threshold_image_filter(smoothed,
                                                 inside_value=0,
                                                 outside_value=1)

connected_components = itk.connected_component_image_filter(thresholded)

In [ ]:
view(label_image=connected_components)

## ipywidgets
http://10.16.11.84:8889/lab/tree/example/vanilla_visualization.ipynb

In [52]:
import SimpleITK as sitk
%matplotlib inline
import matplotlib.pyplot as plt
# import gui
from ipywidgets import interact, fixed
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.colors as colors

In [62]:
def display_with_overlay(segmentation_number, slice_number, image, seg, window_min, window_max, opacity=1):
    """
    Display a slice with segmented contours overlaid into it. 
    """
    img_x = image.GetSize()[0]
    img_y = image.GetSize()[1]
    seg_x = seg.GetSize()[0]
    seg_y = seg.GetSize()[1]
    img = image[int(np.around(img_x/2)):img_x, int(np.around(img_y/2)):img_y, slice_number] # right eye
    seg = seg[int(np.around(seg_x/2)):seg_x, int(np.around(seg_y/2)):seg_y, slice_number] # right eye

    palette_lens = colors.ListedColormap(['red'])
    palette_globe = colors.ListedColormap(['lime'])
    palette_nerve = colors.ListedColormap(['blue'])
    palette_int_fat = colors.ListedColormap(['yellow'])
    palette = [palette_lens, palette_globe, palette_nerve, palette_int_fat]

    if segmentation_number != 0:
        msk = seg[:,:]==segmentation_number
    else:
        msk = seg[:,:]
    overlay_img = sitk.LabelMapOverlay(sitk.Cast(sitk.Cast(msk, sitk.sitkUInt8), sitk.sitkLabelUInt8),
        sitk.Cast(sitk.IntensityWindowing(img, windowMinimum=window_min, windowMaximum=window_max), sitk.sitkUInt8),
        opacity=opacity)
        
    # Assume the original slice is isotropic, otherwise the display would be distorted 
    plt.imshow(sitk.GetArrayViewFromImage(overlay_img), origin='lower')
    plt.axis('off')
    plt.show()

In [41]:
import numpy as np

ima_file = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/non_labeled_dataset_nifti_nnunet/AEye_2022160100001_0000.nii.gz'
lab_file = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing/AEye_2022160100001.nii.gz'

img = sitk.ReadImage(ima_file)
lab = sitk.ReadImage(lab_file)

256


In [69]:
# Example with the segmentations
slices = np.array([87, 88, 89]) # example for subject 001
interact(display_with_overlay, segmentation_number=1,
    slice_number = widgets.IntSlider(min=slices[0], max=slices[-1], step=1, value=slices[0]), 
    image = fixed(img), seg=fixed(lab), window_min=fixed(0), window_max=fixed(4000))

interactive(children=(IntSlider(value=1, description='segmentation_number', max=3, min=-1), IntSlider(value=87…

<function __main__.display_with_overlay(segmentation_number, slice_number, image, seg, window_min, window_max, opacity=1)>